In [9]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import time

import sys
ROOT = "../../"
sys.path.append(ROOT) # Add root directory to path

from src.utils.REPClassifier import REPClassifier
from src.utils.StandardDataset import StandardDataset
from src.utils.perturbations import *

from art.attacks.evasion import FastGradientMethod
from art.estimators.classification import PyTorchClassifier
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, TensorDataset, Dataset

In [10]:
# Step 1: Load the CIFAR-100 dataset
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())
testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms.ToTensor())
num_classes = 10

Files already downloaded and verified
Files already downloaded and verified


In [11]:
x_train = trainset.data / 255.0  # Scale pixel values to [0, 1]
y_train = np.array(trainset.targets)
x_test = testset.data / 255.0  # Scale pixel values to [0, 1]
y_test = np.array(testset.targets)

x_train = np.transpose(x_train, (0, 3, 1, 2)).astype(np.float32)
x_test = np.transpose(x_test, (0, 3, 1, 2)).astype(np.float32)

y_train = np.eye(num_classes)[y_train].astype(np.float32)
y_test = np.eye(num_classes)[y_test].astype(np.float32)

min_pixel_value = 0.0
max_pixel_value = 1.0

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

x_train = torch.from_numpy(x_train).to(device)
y_train = torch.from_numpy(y_train).to(device)

x_test = torch.from_numpy(x_test).to(device)
y_test = torch.from_numpy(y_test).to(device)

In [13]:
class SimpleDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

train_dataset = SimpleDataset(x_train, y_train)
test_dataset = SimpleDataset(x_test, y_test)

# Testing Round 1

In [14]:
epochs = 100
test_history = []

def display_test_history(test_history):
    for i in range(len(test_history)):
        test = test_history[i]
        num_p = len(test['perturbations'])
        io = test['include_original']
        shuffle = test['shuffle']
        baseline = test['baseline_accuracy']
        adversarial = test['adversarial_accuracy']
        adv_str = f"{adversarial[0]*100}/{adversarial[1]*100}/{adversarial[2]*100}/{adversarial[3]*100}"
        print(f"Test {i}: Perturbations ({num_p}), Original ({io}), Shuffle ({shuffle}), Baseline ({baseline*100}%), Adversarial ({adv_str}%)")

In [15]:
from tqdm.auto import tqdm

def train_REPClassifier(model, dataset, optimizer, criterion, batch_size, epochs, verbose=True):
    dataloader = DataLoader(dataset, batch_size, shuffle=True, num_workers=0)
    for _ in tqdm(range(epochs), disable=not verbose, desc="Epochs"):
        for x_batch, y_batch in dataloader:            
            # Zero the parameter gradients
            optimizer.zero_grad()

            # Perform prediction
            model_outputs = model(x_batch)

            y_batch = y_batch.squeeze(1)
            predictions = model_outputs.squeeze(1)
            
            loss = criterion(predictions, y_batch)

            # Do training
            loss.backward()

            optimizer.step()

In [22]:
perturbation_tests = [
    [],
    [["Gaussian", 0.025]],
    [["Gaussian", 0.05]],
    [[["Gaussian", 0.05], ["ContrastBrightness", 0.9, 0.9]]],
    [[["Gaussian", 0.05], ["RotationFlip", 0.3]]],
    [[["Gaussian", 0.05], ["SaltPepper", 0.05]]],
    [["Laplacian", 0.025]],
    [["Laplacian", 0.05]],
    [[["Laplacian", 0.05], ["ContrastBrightness", 0.9, 0.9]]],
    [[["Laplacian", 0.05], ["RotationFlip", 0.3]]],
    [[["Laplacian", 0.05], ["SaltPepper", 0.05]]],
    [["Lp-norm", 0.5, 1]],
    [[["Lp-norm", 0.5, 1], ["ContrastBrightness", 0.9, 0.9]]],
    [[["Lp-norm", 0.5, 1], ["RotationFlip", 0.3]]],
    [[["Lp-norm", 0.5, 1], ["SaltPepper", 0.05]]],
    [["Lp-norm", 0.5, 2]],
    [["Lp-norm", 0.5, 3]],
    [["Lp-norm", 0.5, 4]],
    [["Lp-norm", 0.5, float('inf')]],
    [["ContrastBrightness", 0.9, 0.9]],
    [["RotationFlip", 0.3]],
    [["SaltPepper", 0.05]],
    [["Gaussian", 0.05], ["Laplacian", 0.05]],
    [[["Gaussian", 0.05], ["Laplacian", 0.05]]],
    [["Gaussian", 0.05], ["Lp-norm", 0.5, 1]],
    [["Laplacian", 0.05], ["Lp-norm", 0.5, 1]],
    [["Lp-norm", 0.5, 1], ["Lp-norm", 0.5, 2]],
    [["Gaussian", 0.05], ["Laplacian", 0.05], ["Lp-norm", 0.5, 1]],
    [["Gaussian", 0.05], [["Laplacian", 0.05], ["SaltPepper", 0.05]]],
    [["Gaussian", 0.05], [["Lp-norm", 0.5, 1], ["ContrastBrightness", 0.9, 0.9]]],
    [["Laplacian", 0.05], [["Lp-norm", 0.5, 1], ["RotationFlip", 0.3]]],
    [[["Gaussian", 0.05], ["SaltPepper", 0.05]], [["Gaussian", 0.05], ["RotationFlip", 0.3]]],
    [[["Laplacian", 0.05], ["SaltPepper", 0.05]], [["Laplacian", 0.05], ["RotationFlip", 0.3]]],
    [[["Lp-norm", 0.5, 1], ["SaltPepper", 0.05]], [["Lp-norm", 0.5, 1], ["RotationFlip", 0.3]]],
    [[["Gaussian", 0.05], ["SaltPepper", 0.05]], [["Gaussian", 0.05], ["RotationFlip", 0.3]], [["Lp-norm", 0.5, 1], ["SaltPepper", 0.05]], [["Lp-norm", 0.5, 1], ["RotationFlip", 0.3]]],
    [[["Gaussian", 0.05], ["SaltPepper", 0.05]], [["Gaussian", 0.05], ["RotationFlip", 0.3]], [["Lp-norm", 0.5, 1], ["SaltPepper", 0.05]], [["Lp-norm", 0.5, 1], ["RotationFlip", 0.3]], [["Laplacian", 0.05], ["SaltPepper", 0.05]], [["Laplacian", 0.05], ["RotationFlip", 0.3]]],
    [[["Laplacian", 0.05], ["RotationFlip", 0.3]],[["Gaussian", 0.05], ["RotationFlip", 0.3]],[["Gaussian", 0.05], ["SaltPepper", 0.05]],[["Gaussian", 0.05], ["SaltPepper", 0.05]]],
    [[["Laplacian", 0.05], ["RotationFlip", 0.3]],[["Gaussian", 0.05], ["RotationFlip", 0.3]],[["Gaussian", 0.05], ["SaltPepper", 0.05]],[["Gaussian", 0.05], ["SaltPepper", 0.05]]],[[["Laplacian", 0.025], ["RotationFlip", 0.3]],[["Gaussian", 0.025], ["RotationFlip", 0.3]],[["Gaussian", 0.025], ["SaltPepper", 0.05]],[["Gaussian", 0.025], ["SaltPepper", 0.05]]],
]

In [23]:
channel_widths = [3, 50, 100, 150, 100, 50]  
linear_sizes = [50, 100]
kernel = 3
pooling = nn.AdaptiveMaxPool2d((2, 2))
lr = 0.0001
criterion = nn.CrossEntropyLoss()

for perturbation_description in perturbation_tests:
    perturbations = create_perturbations(perturbation_description)
    include_original = perturbations == []
    shuffle = True

    model = REPClassifier(
        channel_widths=channel_widths,
        linear_sizes=linear_sizes,
        kernel=kernel,
        pooling=pooling,
        perturbations=perturbations,
        include_original=include_original,
        shuffle=shuffle,
        num_classes=num_classes,
    )
    
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=0.0001)
    batch_size = 64
    super_epochs = 4
    epochs_per = 20
    print(f"Training model with perturbations:")
    print(perturbation_description)
    training_start = time.time()
    for epoch in range(super_epochs):
        train_REPClassifier(model, train_dataset, optimizer, criterion, batch_size, epochs_per, True)
        with torch.no_grad():
            train_accuracy = torch.sum(torch.argmax(model(x_train[:1200]), axis=1) == torch.argmax(y_train[:1200], axis=1)) / len(x_train[:1200])
            test_accuracy = torch.sum(torch.argmax(model(x_test[:1200]), axis=1) == torch.argmax(y_test[:1200], axis=1)) / len(x_test[:1200])
            print(f"Epoch {(epoch+1)*epochs_per}/{super_epochs*epochs_per} Complete After {int(time.time() - training_start)}s! Train Acc: {train_accuracy}, Test Acc: {test_accuracy}")
    
    classifier = PyTorchClassifier(
        model=model,
        clip_values=(min_pixel_value, max_pixel_value),
        loss=criterion,
        optimizer=optimizer,
        input_shape=(3, 28, 28),
        nb_classes=num_classes,
    )

    x_test_numpy = x_test.cpu().numpy()
    y_test_numpy = y_test.cpu().numpy()
    with torch.no_grad():
        # Step 5: Evaluate the ART classifier on benign test examples
        predictions = classifier.predict(x_test_numpy)
        baseline_accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test_numpy, axis=1)) / len(y_test)
        print("Accuracy on benign test examples: {}%".format(baseline_accuracy * 100))

    # Step 6: Generate adversarial test examples
    attack = FastGradientMethod(estimator=classifier, eps=0.025)
    x_test_adv1 = attack.generate(x=x_test_numpy)
    with torch.no_grad():
        predictions = classifier.predict(x_test_adv1)
        adversarial_accuracy1 = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test_numpy, axis=1)) / len(y_test)
        print("Accuracy on adversarial test examples (epsilon = 0.025): {}%".format(adversarial_accuracy1 * 100))

    attack = FastGradientMethod(estimator=classifier, eps=0.05)
    x_test_adv2 = attack.generate(x=x_test_numpy)
    with torch.no_grad():
        predictions = classifier.predict(x_test_adv2)
        adversarial_accuracy2 = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test_numpy, axis=1)) / len(y_test)
        print("Accuracy on adversarial test examples (epsilon = 0.05): {}%".format(adversarial_accuracy2 * 100))

    attack = FastGradientMethod(estimator=classifier, eps=0.075)
    x_test_adv3 = attack.generate(x=x_test_numpy)
    with torch.no_grad():
        predictions = classifier.predict(x_test_adv3)
        adversarial_accuracy3 = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test_numpy, axis=1)) / len(y_test)
        print("Accuracy on adversarial test examples (epsilon = 0.075): {}%".format(adversarial_accuracy3 * 100))

    attack = FastGradientMethod(estimator=classifier, eps=0.1)
    x_test_adv4 = attack.generate(x=x_test_numpy)
    with torch.no_grad():
        predictions = classifier.predict(x_test_adv4)
        adversarial_accuracy4 = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test_numpy, axis=1)) / len(y_test)
        print("Accuracy on adversarial test examples (epsilon = 0.1): {}%".format(adversarial_accuracy4 * 100))

    test_history.append({'channel_widths': channel_widths,
                        'linear_sizes': linear_sizes,
                        'kernel': kernel,
                        'pooling': pooling,
                        'perturbations': perturbation_description,
                        'include_original': include_original,
                        'shuffle': shuffle,
                        'loss': criterion,
                        'lr': lr,
                        'batch_size': batch_size,
                        'epochs': epochs,
                        'baseline_accuracy': baseline_accuracy,
                        'adversarial_accuracy': [adversarial_accuracy1, adversarial_accuracy2, adversarial_accuracy3, adversarial_accuracy4]})

Training model with perturbations:
[]


Epochs:   0%|          | 0/20 [00:00<?, ?it/s]

Epoch 20/80 Complete After 152s! Train Acc: 0.6324999928474426, Test Acc: 0.6066666841506958


Epochs:   0%|          | 0/20 [00:00<?, ?it/s]

Epoch 40/80 Complete After 305s! Train Acc: 0.7191666960716248, Test Acc: 0.6708333492279053


...

Epoch 40/80 Complete After 3347s! Train Acc: 0.6141666769981384, Test Acc: 0.6616666913032532


Epochs:   0%|          | 0/20 [00:00<?, ?it/s]

Epoch 60/80 Complete After 5028s! Train Acc: 0.7400000095367432, Test Acc: 0.7433333396911621


Epochs:   0%|          | 0/20 [00:00<?, ?it/s]

Epoch 80/80 Complete After 6719s! Train Acc: 0.7516666650772095, Test Acc: 0.7275000214576721
Accuracy on benign test examples: 71.63000000000001%
Accuracy on adversarial test examples (epsilon = 0.025): 27.49%
Accuracy on adversarial test examples (epsilon = 0.05): 16.7%
Accuracy on adversarial test examples (epsilon = 0.075): 14.149999999999999%
Accuracy on adversarial test examples (epsilon = 0.1): 13.120000000000001%


In [24]:
idx = 10

plt.figure()
plt.imshow(np.transpose(x_test[idx], (1, 2, 0)))
plt.show()
# print(np.argmax(classifier.predict(x_test[idx]), axis=1))

plt.figure()
plt.imshow(np.transpose(x_test_adv1[idx], (1, 2, 0)))
plt.show()
# print(np.argmax(classifier.predict(x_test_adv2[idx]), axis=1))


TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

<Figure size 640x480 with 0 Axes>

In [25]:
display_test_history(test_history)

Test 0: Perturbations (0), Original (True), Shuffle (True), Baseline (71.12%), Adversarial (15.129999999999999/13.84/12.72/11.3%)
Test 1: Perturbations (1), Original (False), Shuffle (True), Baseline (71.77%), Adversarial (15.52/13.600000000000001/12.27/11.0%)
Test 2: Perturbations (1), Original (False), Shuffle (True), Baseline (71.95%), Adversarial (19.88/13.63/12.520000000000001/11.379999999999999%)
Test 3: Perturbations (1), Original (False), Shuffle (True), Baseline (56.28999999999999%), Adversarial (36.8/27.74/26.0/25.91%)
Test 4: Perturbations (1), Original (False), Shuffle (True), Baseline (66.67999999999999%), Adversarial (44.45/31.009999999999998/23.53/19.950000000000003%)
Test 5: Perturbations (1), Original (False), Shuffle (True), Baseline (67.60000000000001%), Adversarial (40.300000000000004/22.09/16.400000000000002/14.75%)
Test 6: Perturbations (1), Original (False), Shuffle (True), Baseline (72.0%), Adversarial (17.26/13.62/11.87/11.31%)
Test 7: Perturbations (1), Origin

In [60]:
best_i = 0
accuracy = 0
eps = 3
for i in range(len(test_history)):
    if test_history[i]['adversarial_accuracy'][eps] > accuracy:
        best_i = i
        accuracy = test_history[i]['adversarial_accuracy'][eps]
print(test_history[best_i])

{'channel_widths': [3, 50, 100, 50], 'linear_sizes': [50, 100], 'kernel': 3, 'pooling': AdaptiveMaxPool2d(output_size=(2, 2)), 'perturbations': [['Lp-norm', 1, inf]], 'include_original': False, 'shuffle': True, 'loss': CrossEntropyLoss(), 'lr': 0.0001, 'batch_size': 64, 'epochs': 50, 'baseline_accuracy': 0.2448, 'adversarial_accuracy': [0.0879, 0.0369, 0.0226, 0.0134]}


In [26]:
import csv

def save_to_csv(data, filename):
    if not data:
        return

    # Extract the keys from the first dictionary as headers
    headers = list(data[0].keys())

    with open(filename, 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=headers)

        # Write the headers
        writer.writeheader()

        # Write each dictionary as a row in the CSV file
        for item in data:
            writer.writerow(item)

    print(f"Data saved to {filename} successfully.")

save_to_csv(test_history, ROOT + "notebooks/experiment_log/cifar_10_CNN_round_1_training_results.csv")

Data saved to ../../notebooks/experiment_log/cifar_10_CNN_round_1_training_results.csv successfully.


# Testing Round 2

In [20]:
perturbation_tests = [
    # [],
    # [["Gaussian", 0.025]],
    # [["Gaussian", 0.05]],
    [[["Gaussian", 0.05], ["ContrastBrightness", 0.9, 0.9]]],
    [[["Gaussian", 0.05], ["RotationFlip", 0.3]]],
    [[["Gaussian", 0.05], ["SaltPepper", 0.05]]],
    [["Laplacian", 0.025]],
    [["Laplacian", 0.05]],
    [[["Laplacian", 0.05], ["ContrastBrightness", 0.9, 0.9]]],
    [[["Laplacian", 0.05], ["RotationFlip", 0.3]]],
    [[["Laplacian", 0.05], ["SaltPepper", 0.05]]],
    [["Lp-norm", 0.5, 1]],
    [[["Lp-norm", 0.5, 1], ["ContrastBrightness", 0.9, 0.9]]],
    [[["Lp-norm", 0.5, 1], ["RotationFlip", 0.3]]],
    [[["Lp-norm", 0.5, 1], ["SaltPepper", 0.05]]],
    [["Lp-norm", 0.5, 2]],
    [["Lp-norm", 0.5, 3]],
    [["Lp-norm", 0.5, 4]],
    [["Lp-norm", 0.5, float('inf')]],
    [["ContrastBrightness", 0.9, 0.9]],
    [["RotationFlip", 0.3]],
    [["SaltPepper", 0.05]],
    [["Gaussian", 0.05], ["Laplacian", 0.05]],
    [[["Gaussian", 0.05], ["Laplacian", 0.05]]],
    [["Gaussian", 0.05], ["Lp-norm", 0.5, 1]],
    [["Laplacian", 0.05], ["Lp-norm", 0.5, 1]],
    [["Lp-norm", 0.5, 1], ["Lp-norm", 0.5, 2]],
    [["Gaussian", 0.05], ["Laplacian", 0.05], ["Lp-norm", 0.5, 1]],
    [["Gaussian", 0.05], [["Laplacian", 0.05], ["SaltPepper", 0.05]]],
    [["Gaussian", 0.05], [["Lp-norm", 0.5, 1], ["ContrastBrightness", 0.9, 0.9]]],
    [["Laplacian", 0.05], [["Lp-norm", 0.5, 1], ["RotationFlip", 0.3]]],
]

In [21]:
test_history = []

def display_test_history(test_history):
    for i in range(len(test_history)):
        test = test_history[i]
        num_p = len(test['perturbations'])
        io = test['include_original']
        shuffle = test['shuffle']
        training_time = test['training_time']
        baseline = test['baseline_accuracy']
        adversarial = test['adversarial_accuracy']
        adv_str = f"{adversarial[0]*100}/{adversarial[1]*100}/{adversarial[2]*100}/{adversarial[3]*100}"
        print(f"Test {i}: Perturbations ({num_p}), Original ({io}), Shuffle ({shuffle}), Training Time ({training_time}s), Baseline ({baseline*100}%), Adversarial ({adv_str}%)")

In [22]:
channel_widths = [3, 50, 100, 50]  
linear_sizes = [50, 100]
kernel = 3
pooling = nn.AdaptiveMaxPool2d((2, 2))
lr = 0.0001
criterion = nn.CrossEntropyLoss()

for perturbation_description in perturbation_tests:
    perturbations = create_perturbations(perturbation_description)
    include_original = perturbation_description == []
    shuffle = True

    model = REPClassifier(
        channel_widths=channel_widths,
        linear_sizes=linear_sizes,
        kernel=kernel,
        pooling=pooling,
        perturbations=perturbations,
        include_original=include_original,
        shuffle=shuffle,
        num_classes=100,
    )
    
    optimizer = optim.Adam(model.parameters(), lr=lr)

    classifier = PyTorchClassifier(
        model=model,
        clip_values=(min_pixel_value, max_pixel_value),
        loss=criterion,
        optimizer=optimizer,
        input_shape=(3, 32, 32),
        nb_classes=100,
    )

    batch_size = 64
    super_epochs = 10
    epochs_per = 10
    print(f"Training model with perturbations:")
    print(perturbation_description)
    training_start = time.time()
    for epoch in range(super_epochs):
        classifier.fit(x_train, y_train, batch_size=batch_size, nb_epochs=epochs_per, verbose=False)
        with torch.no_grad():
            train_predictions = classifier.predict(x_train)
            train_accuracy = np.sum(np.argmax(train_predictions, axis=1) == np.argmax(y_train, axis=1)) / len(y_train)
            test_predictions = classifier.predict(x_test)
            test_accuracy = np.sum(np.argmax(test_predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
            print(f"Epoch {(epoch+1)*epochs_per}/{super_epochs*epochs_per} Complete After {int(time.time() - training_start)}s! Train Acc: {train_accuracy}, Test Acc: {test_accuracy}")
    total_time = time.time() - training_start

    # Step 5: Evaluate the ART classifier on benign test examples
    predictions = classifier.predict(x_test)
    baseline_accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
    print("Accuracy on benign test examples: {}%".format(baseline_accuracy * 100))

    # Step 6: Generate adversarial test examples
    attack = FastGradientMethod(estimator=classifier, eps=0.025)
    x_test_adv1 = attack.generate(x=x_test)
    predictions = classifier.predict(x_test_adv1)
    adversarial_accuracy1 = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
    print("Accuracy on adversarial test examples (epsilon = 0.0.025): {}%".format(adversarial_accuracy1 * 100))

    attack = FastGradientMethod(estimator=classifier, eps=0.05)
    x_test_adv2 = attack.generate(x=x_test)
    predictions = classifier.predict(x_test_adv2)
    adversarial_accuracy2 = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
    print("Accuracy on adversarial test examples (epsilon = 0.05): {}%".format(adversarial_accuracy2 * 100))

    attack = FastGradientMethod(estimator=classifier, eps=0.075)
    x_test_adv3 = attack.generate(x=x_test)
    predictions = classifier.predict(x_test_adv3)
    adversarial_accuracy3 = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
    print("Accuracy on adversarial test examples (epsilon = 0.075): {}%".format(adversarial_accuracy3 * 100))

    attack = FastGradientMethod(estimator=classifier, eps=0.1)
    x_test_adv4 = attack.generate(x=x_test)
    predictions = classifier.predict(x_test_adv4)
    adversarial_accuracy4 = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
    print("Accuracy on adversarial test examples (epsilon = 0.1): {}%".format(adversarial_accuracy4 * 100))

    test_history.append({'channel_widths': channel_widths,
                        'linear_sizes': linear_sizes,
                        'kernel': kernel,
                        'pooling': pooling,
                        'perturbations': perturbation_description,
                        'include_original': include_original,
                        'shuffle': shuffle,
                        'loss': criterion,
                        'lr': lr,
                        'batch_size': batch_size,
                        'epochs': int(super_epochs*epochs_per),
                        'training_time': total_time,
                        'baseline_accuracy': baseline_accuracy,
                        'adversarial_accuracy': [adversarial_accuracy1, adversarial_accuracy2, adversarial_accuracy3, adversarial_accuracy4]})

Training model with perturbations:
[[['Gaussian', 0.05], ['ContrastBrightness', 0.9, 0.9]]]
Epoch 10/100 Complete After 53s! Train Acc: 0.06036, Test Acc: 0.0611
Epoch 20/100 Complete After 107s! Train Acc: 0.09602, Test Acc: 0.1031
Epoch 30/100 Complete After 161s! Train Acc: 0.12176, Test Acc: 0.1243
Epoch 40/100 Complete After 215s! Train Acc: 0.14082, Test Acc: 0.1438
Epoch 50/100 Complete After 270s! Train Acc: 0.16166, Test Acc: 0.1649
Epoch 60/100 Complete After 325s! Train Acc: 0.17782, Test Acc: 0.1713
Epoch 70/100 Complete After 381s! Train Acc: 0.18964, Test Acc: 0.1919
Epoch 80/100 Complete After 436s! Train Acc: 0.20538, Test Acc: 0.1937
Epoch 90/100 Complete After 491s! Train Acc: 0.21622, Test Acc: 0.1929
Epoch 100/100 Complete After 547s! Train Acc: 0.2379, Test Acc: 0.2216
Accuracy on benign test examples: 22.53%
Accuracy on adversarial test examples (epsilon = 0.0.025): 15.690000000000001%
Accuracy on adversarial test examples (epsilon = 0.05): 12.35%
Accuracy on adve

In [23]:
display_test_history(test_history=test_history)

Test 0: Perturbations (1), Original (False), Shuffle (True), Training Time (547.7024054527283s), Baseline (22.53%), Adversarial (15.690000000000001/12.35/10.96/9.54%)
Test 1: Perturbations (1), Original (False), Shuffle (True), Training Time (544.7114560604095s), Baseline (33.410000000000004%), Adversarial (21.41/12.47/7.8/5.27%)
Test 2: Perturbations (1), Original (False), Shuffle (True), Training Time (554.6939740180969s), Baseline (32.72%), Adversarial (22.05/13.34/9.21/6.94%)
Test 3: Perturbations (1), Original (False), Shuffle (True), Training Time (728.2032270431519s), Baseline (37.21%), Adversarial (12.22/7.140000000000001/5.1/3.6799999999999997%)
Test 4: Perturbations (1), Original (False), Shuffle (True), Training Time (729.4358289241791s), Baseline (35.370000000000005%), Adversarial (16.09/9.27/6.2700000000000005/4.4799999999999995%)
Test 5: Perturbations (1), Original (False), Shuffle (True), Training Time (750.4908287525177s), Baseline (21.11%), Adversarial (17.04/11.77/10.

In [24]:
import csv

def save_to_csv(data, filename):
    if not data:
        return

    # Extract the keys from the first dictionary as headers
    headers = list(data[0].keys())

    with open(filename, 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=headers)

        # Write the headers
        writer.writeheader()

        # Write each dictionary as a row in the CSV file
        for item in data:
            writer.writerow(item)

    print(f"Data saved to {filename} successfully.")

save_to_csv(test_history, ROOT + "notebooks/experiment_log/cifar_100_round_2_training_results.csv")

Data saved to ../../notebooks/experiment_log/cifar_100_round_2_training_results.csv successfully.
